# **Использование абстракция для решения задач**

Решим простую задачу о повороте треугольника ABC с использованием следующих перестановок:  
a = (A B C -> B C A)  
b = (A B C -> C A B)  
e = (A B C -> A B C),  
где A, B, C - вершины треугольника ABC.  
Нужно составить таблицу, где пересечением элементов является результат перемножения перестановок.  


|   | e  | a  | b  |
| --|:--:| :--:| :--:|
| e |     |    |    |
| a |     |    |  e |
| b |     |    |    |

[Что такое перестановки и как их перемножать](http://halgebra.math.msu.su/wiki/lib/exe/fetch.php/permutations.pdf).

В начале обозначим наши вершины и перестановки, как перечисление и функции соответственно.

In [1]:
from enum import Enum
class Points(Enum):
    A = 1
    B = 2
    C = 3
globals().update(Points.__members__)

In [2]:
def a(a, b, c): return b, c, a
def b(a, b, c): return c, a, b
def e(a, b, c): return a, b, c

In [3]:
permuts1 = [e, a, b]

In [4]:
for p in permuts1:
    for p1 in permuts1:
        rs = p(*p1(A, B, C))
        print(#p.__name__, '*', p1.__name__ , '=', 
              'a' if rs == a(A,B,C) else '', 
              'b' if rs == b(A,B,C) else '', 
              'e' if rs == e(A,B,C) else '', 
              sep='', end=' ')
    print()

e a b 
a b e 
b e a 


In [5]:
def c(a, b, c): return a, c, b
def d(a, b, c): return c, b, a
def f(a, b, c): return b, a, c

In [6]:
permuts2 = [e, a, b, c, d, f]

In [7]:
for p in permuts2:
    for p1 in permuts2:
        rs = p1(*p(A, B, C))
        print(#p.__name__, '*', p1.__name__ , '=', 
              'a' if rs == a(A,B,C) else '', 
              'b' if rs == b(A,B,C) else '', 
              'c' if rs == c(A,B,C) else '', 
              'd' if rs == d(A,B,C) else '', 
              'f' if rs == f(A,B,C) else '', 
              'e' if rs == e(A,B,C) else '', 
              sep='', end=' ')
    print()

e a b c d f 
a b e f c d 
b e a d f c 
c d f e a b 
d f c b e a 
f c d a b e 


Однако раскрывать кортеж не всегда удобно, особенно, если нужно перестановок много, попробуем использовать декоратор.

In [8]:
from functools import wraps

def a_w(p):
    @wraps(p)
    def wrapper(a, b, c):
         return p(b, c, a)
    return wrapper

def b_w(p):
    @wraps(p)
    def wrapper(a, b, c):
         return p(c, a, b)
    return wrapper


def e_w(p):
    @wraps(p)
    def wrapper(a, b, c):
         return p(a, b, c)
    return wrapper

b(A, B, C) == e_w(b)(A, B, C)

True

Заметим, что при увеличении количества перемножений чтение кода все равно затрудняется

In [9]:
a_w(e_w(b_w(b)))(A, B, C) == a_w(a)(A, B, C)

True

Добавим немножко ООП и перегруженных операторов:

In [10]:
class Permutation:
    
    def __init__(self, p):
        self.p = p
    
    def __mul__(self, rhs):
        if type(self) == Permutation:
            lhsp = self.p
        if type(rhs) == Permutation:
            rhsp = rhs.p
        
        def wrapper(a, b, c):
            return rhsp(*lhsp(a, b, c))
        
        np = Permutation(wrapper)
        
        return np
        
    def __call__(self, *args):
        return self.p(*args)
    
    def __eq__(self, rhs):
        return self.p(A, B, C) == rhs(A, B, C)
        
a = Permutation(lambda a, b, c: (b, c, a))
b = Permutation(lambda a, b, c: (c, a, b))
e = Permutation(lambda a, b, c: (a, b, c))

In [11]:
a == e

False

С помощью оператора __eq__ мы даже можем узнать равны ли перестановки друг другу:

In [12]:
a*a == b

True